# Simulate

This example gives a "hello world" example to use AMS.

## Import and Setting the Verbosity Level

We first import the `ams` library.

In [1]:
import ams

import datetime

In [2]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f'ams:{ams.__version__}')

Last run time: 2024-11-24 17:46:14
ams:0.9.12


We can configure the verbosity level for logging (output messages) by passing a verbosity level (10-DEBUG, 20-INFO, 30-WARNING, 40-ERROR, 50-CRITICAL) to the `stream_level` argument of `ams.main.config_logger()`. Verbose level 10 is useful for getting debug output.

The logging level can be altered by calling ``config_logger`` again with new ``stream_level`` and ``file_level``.

In [3]:
ams.config_logger(stream_level=20)

Note that the above ``ams.config_logger()`` is a shorthand to ``ams.main.config_logger()``.

If this step is omitted, the default `INFO` level (`stream_level=20`) will be used.

## Run Simulations

### Load Case

AMS supports multiple input file formats, including AMS ``.xlsx`` file, MATPOWER ``.m`` file, PYPOWER ``.py`` file, and PSS/E ``.raw`` file.

Here we use the AMS ``.xlsx`` file as an example. The source file locates at ``$HOME/ams/ams/cases/ieee39/ieee39_uced.xlsx``.

In [4]:
sp = ams.load(ams.get_case('5bus/pjm5bus_uced.xlsx'),
              setup=True,
              no_output=True,)

Working directory: "/Users/jinningwang/work/ams/examples"
Parsing input file "/Users/jinningwang/work/miniconda3/envs/amsre/lib/python3.12/site-packages/ams/cases/5bus/pjm5bus_uced.xlsx"...
Input file parsed in 0.0231 seconds.
Zero line rates detacted in rate_b, rate_c, adjusted to 999.
System set up in 0.0022 seconds.


### Inspect Models and Routines

In AMS, ``model`` refers to the device-level models, and they are registered to an OrderedDict ``models``.

In [5]:
sp.models

OrderedDict([('Summary', Summary (3 devices) at 0x1177b4aa0),
             ('Bus', Bus (5 devices) at 0x30dfb8260),
             ('PQ', PQ (3 devices) at 0x30dfbbdd0),
             ('Slack', Slack (1 device) at 0x30ecaa0c0),
             ('PV', PV (3 devices) at 0x30f8544a0),
             ('Shunt', Shunt (0 devices) at 0x30efc8890),
             ('Line', Line (7 devices) at 0x30df930e0),
             ('Jumper', Jumper (0 devices) at 0x30f855ca0),
             ('PVD1', PVD1 (0 devices) at 0x30e3116a0),
             ('ESD1', ESD1 (0 devices) at 0x30f8568a0),
             ('EV1', EV1 (0 devices) at 0x30ef49370),
             ('EV2', EV2 (0 devices) at 0x30f857350),
             ('REGCA1', REGCA1 (0 devices) at 0x30f857770),
             ('REGCV1', REGCV1 (0 devices) at 0x30f857ef0),
             ('REGCV2', REGCV2 (0 devices) at 0x30f8945f0),
             ('Area', Area (3 devices) at 0x30f894ad0),
             ('Region', Region (2 devices) at 0x30f894fe0),
             ('SFR', SFR (2 devic

One can inspect the detailed model data in the form of DataFrame.

In [6]:
sp.PQ.as_df()

,idx,u,name,bus,Vn,p0,q0,vmax,vmin,owner,ctrl
uid,,,,,,,,,,,
0,PQ_1,1.0,PQ 1,Bus_2,230.0,3.0,0.9861,1.1,0.9,None,1.0
1,PQ_2,1.0,PQ 2,Bus_3,230.0,3.0,0.9861,1.1,0.9,None,1.0
2,PQ_3,1.0,PQ 3,Bus_4,230.0,4.0,1.3147,1.1,0.9,None,1.0


In AMS, all supported routines are registered to an OrderedDict ``routines``.

In [7]:
sp.routines

OrderedDict([('DCPF', DCPF at 0x1177b48f0),
             ('PFlow', PFlow at 0x30f8e0e60),
             ('CPF', CPF at 0x30f8e1850),
             ('ACOPF', ACOPF at 0x30f917e30),
             ('DCOPF', DCOPF at 0x30df2b6e0),
             ('ED', ED at 0x30dfba270),
             ('EDDG', EDDG at 0x30f9656a0),
             ('EDES', EDES at 0x30f9672c0),
             ('RTED', RTED at 0x30f9b18e0),
             ('RTEDDG', RTEDDG at 0x30f9b2d20),
             ('RTEDES', RTEDES at 0x30f9e8380),
             ('RTEDVIS', RTEDVIS at 0x30f9ea2a0),
             ('UC', UC at 0x30f9ebad0),
             ('UCDG', UCDG at 0x30fa26810),
             ('UCES', UCES at 0x30fa5c920),
             ('DOPF', DOPF at 0x30fa5f4a0),
             ('DOPFVIS', DOPFVIS at 0x30fa8ce30),
             ('PFlow0', PFlow0 at 0x30fa8dca0),
             ('DCPF0', DCPF0 at 0x30fa8e450)])

### Solve a Routine

Before solving a routine, we need to initialize it first.
Here Real-time Economic Dispatch (RTED) is used as an example.

In [8]:
sp.RTED.init()

Building system matrices
Parsing OModel for <RTED>
Evaluating OModel for <RTED>
Finalizing OModel for <RTED>
<RTED> initialized in 0.0107 seconds.


True

Then, one can solve it by calling ``run()``.
Here, argument `solver` can be passed to specify the solver to use, such as `solver='ECOS'`.

Installed solvers can be listed by ``ams.shared.installed_solvers``,
and more detailes of solver can be found at [CVXPY-Choosing a solver](https://www.cvxpy.org/tutorial/advanced/index.html#choosing-a-solver).

In [9]:
ams.shared.installed_solvers

['CLARABEL', 'ECOS', 'ECOS_BB', 'OSQP', 'SCIP', 'SCIPY', 'SCS']

In [10]:
sp.RTED.run(solver='CLARABEL')

<RTED> solved as optimal in 0.0105 seconds, converged in 10 iterations with CLARABEL.


True

The solved results are stored in each variable itself.
For example, the solved power generation of ten generators
are stored in ``pg.v``.

In [11]:
sp.RTED.pg.v

array([2. , 2.1, 5.2, 0.7])

Here, ``get_idx()`` can be used to get the index of a variable.

In [12]:
sp.RTED.pg.get_idx()

['Slack_4', 'PV_1', 'PV_3', 'PV_5']

Part of the solved results can be accessed with given indices.

In [13]:
sp.RTED.get(src='pg', attr='v', idx=['PV_1', 'PV_3'])

array([2.1, 5.2])

All Vars are listed in an OrderedDict ``vars``.

In [14]:
sp.RTED.vars

OrderedDict([('pg', Var: StaticGen.pg),
             ('aBus', Var: Bus.aBus),
             ('pru', Var: StaticGen.pru),
             ('prd', Var: StaticGen.prd)])

The Objective value can be accessed with ``obj.v``.

In [15]:
sp.RTED.obj.v

0.19537500002212937

Similarly, all Constrs are listed in an OrderedDict ``constrs``,
and the expression values can also be accessed.

In [16]:
sp.RTED.constrs

OrderedDict([('pb', Constraint: pb [ON]),
             ('sba', Constraint: sbus [ON]),
             ('pglb', Constraint: pglb [ON]),
             ('pgub', Constraint: pgub [ON]),
             ('plflb', Constraint: plflb [ON]),
             ('plfub', Constraint: plfub [ON]),
             ('alflb', Constraint: alflb [ON]),
             ('alfub', Constraint: alfub [ON]),
             ('rbu', Constraint: rbu [ON]),
             ('rbd', Constraint: rbd [ON]),
             ('rru', Constraint: rru [ON]),
             ('rrd', Constraint: rrd [ON]),
             ('rgu', Constraint: rgu [ON]),
             ('rgd', Constraint: rgd [ON])])

One can also inspect the `Constraint` values.

In [17]:
sp.RTED.rgu.v

array([ -997.    ,  -997.9   ,  -997.0349, -1002.9651])